# Support Vector Machines

_Adapted from Yish's interpretation of Chapter 9 of Introduction to Statistical Learning in R_

## Support Vector Classifier

SVMs approach the classification problem in a direct way - __we try and find a plane that separates the classes in the feature space__

If we cannot, we do one of the two things:

- We __soften__ what we mean by "separates" 
- We __enlarge__ the feature space so that the separation is possible

__Notes on Terminology__:

- Support vector machines are sometimes used as a general method that incorporate maximal margin classifier, support vector classifiers etc. However, strictly by definition, support vector machine is a support vector classifier utilized with non-linear kernel. 

> "When the support vector classifier is combined with a non-linear kernel such as (9.22), the resulting classifier is known as a support vector machine." -- P366, ISLR


Example of a boundary hyperplane in a two dimensional space:

<img src="images/exampleboundary.png" width=500>

### Maximal Margin Classifier

SVM tackles the problem of classification directly, in the sense that it does not compute a probabilistic model. Instead, it constructs a hyperplane to directly separate the classes. 

For example:

![](images/manyboundaries.png)

However, the problem with this approach is that we can come up with infinite number of such hyperplanes as we can tilt the line back and forth and it would still serve the same purpose.

__Therefore, we are using the hyperplane such that the it would be the farthest from training observations from either side__. The intuition behind it is that if we have chosen a hyperplane that is far from the training observations, it would be far for the testing observations as well. <br>

The distance between the training observations and the hyperplane is called the _margin_, and the classifier aims to find the maximal margin from the hyperplane that separates the training examples:

<img src="images/bestboundary.png" width=500>

## Soft Margin Classifier

Even though the maximal margin classifier sounds like an intuitive idea and not too difficult to optimize for, it might not be desirable under two circumstances:

1. It will be sensitive to individual training observations
2. The algorithm will not converge if the training observations cannot be linearly separated.

![](images/softboundary.png)

What happens if we cannot come up with a hyperplane that perfectly separates the training observations like the ones above? The first solution is the soft margin classifier, where we can loosen up our definition of the margin. 

<img src="images/withoutboundary.png" width=500>

__Rather than seeking the largest possible margin so that every observation is not only on the correct side of the hyperplane but also on the correct side of the margin, we instead allow some observations to be on the incorrect side of the margin, or even on the incorrect side of the hyperplane.__

![](images/errortolerance.png)

In this case, the hyperparameter ε is known as the slack variables, which dictate how many training observations we allow to violate the rule of margins or even the hyperplane. The amount of slack is bounded by C accordingly.

The parameter εi tells us where the ith training observation is located. 
- If εi = 0, then we say the ith training observation is on the correct side of the margin;
- If εi > 0, then we say it has violated the margin
- If εi > 1, then it is on the wrong side of the hyperplane

The value C is the sum of ε across all i training observations. The parameter C controls the bias-variance tradeoff of the statistical technique. A high value of C meaning we are more tolerant of the violation, which in turn might give us a model that has high bias but low variance; however, a low value of C indicates low tolerance of the violation, potentially giving us more variance but less bias. 

__How do we determine the ideal value of C?__

![](images/impactofchyperparameter.png)

__Note!__

In scikit-learn implementation, `c` is defined as the inverse. A higher value of `c` is a smaller regularization or smaller penalty, whereas a lower value of `c` is a higher penalty.

__Note!__
It is important to point out that in the support vector classfier (or SVM) in general, only the vectors on the margins are used for classification. They are called the __"Support Vectors"__

## "The Kernel Trick"

Sometimes we have training data that are not able to be separated even with softened margin:

![](images/needforkerneltrick.png)

The intuition to find the optimal fit is called feature space expansion:

- First, we __enlarge__ the feature space through the use of kernel
- Fit a support vector classifier in the enlarged space 
- This results in nonlinear decision boundaries in the original space 


<img src="images/nonlinearring.png" width=500>

<img src="images/nonlinearringin3d.png" width=500>

Why do we know that enlarging the feature space makes the data more linearly separable? [Cover's Theorem](https://en.wikipedia.org/wiki/Cover%27s_theorem).

Another view:

![](images/kernel_trick_hyperdimensional.png)

## Implementation & Performance Comparison

Docs: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  

from sklearn.model_selection import train_test_split  
from time import time

In [ ]:
df = pd.read_csv('data/data_banknote_authentication.csv', header = None)
df.head()

In [ ]:
df.info()

In [ ]:
# our data needs column names
headers = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"]
df.columns = headers

df.head()

In [ ]:
# define X and y, then train test split
X = df.drop('Class', axis=1)  
y = df['Class'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) 

### First: Linear Kernel

In [ ]:
from sklearn.svm import SVC
tic = time() #timing!

svc_linear = SVC(kernel='linear', C=1)
svc_linear.fit(X_train, y_train)

y_pred_train = svc_linear.predict(X_train)
y_pred_test = svc_linear.predict(X_test)

toc = time()
print(f"Run time is {toc-tic} seconds")

In [ ]:
# how'd we do?
from sklearn.metrics import classification_report, plot_confusion_matrix , accuracy_score

print(classification_report(y_test, y_pred_test)) 
print(f"Train accuracy: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.4f}")

plot_confusion_matrix(svc_linear, X_test, y_test)
plt.show()

### Now: RBF

In [ ]:
tic = time() #timing!

svc_rbf = SVC(kernel='rbf', C=1, gamma='scale') # using all default values here
svc_rbf.fit(X_train, y_train)

y_pred_train = svc_rbf.predict(X_train)
y_pred_test = svc_rbf.predict(X_test)

toc = time()
print(f"Run time is {toc-tic} seconds")

In [ ]:
# how'd we do?
print(classification_report(y_test, y_pred_test)) 
print(f"Train accuracy: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.4f}")

plot_confusion_matrix(svc_rbf, X_test, y_test)
plt.show()

### And a Polynomial Kernel for good measure

In [ ]:
tic = time() #timing!

svc_poly = SVC(kernel='poly', C=1, gamma='scale', degree=3) # using mostly default values here
svc_poly.fit(X_train, y_train)

y_pred_train = svc_poly.predict(X_train)
y_pred_test = svc_poly.predict(X_test)

toc = time()
print(f"Run time is {toc-tic} seconds")

In [ ]:
# how'd we do?
print(classification_report(y_test, y_pred_test)) 
print(f"Train accuracy: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.4f}")

plot_confusion_matrix(svc_poly, X_test, y_test)
plt.show()

### Adjusting C?

In [ ]:
for c in [.01, 1, 100]: 
    svc_c = SVC(kernel='linear', C=c, gamma='scale') # going linear again
    svc_c.fit(X_train, y_train)

    y_pred_train = svc_c.predict(X_train)
    y_pred_test = svc_c.predict(X_test)

    # how'd we do?
    print("-----")
    print(f'Results at C = {c}')
    print(classification_report(y_test, y_pred_test)) 
    print(f"Train accuracy: {accuracy_score(y_train, y_pred_train):.4f}")
    print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.4f}")

    plot_confusion_matrix(svc_c, X_test, y_test)
    plt.show()

## Pros 

- Good for datasets with more variables than observations
- Robust against outliers
- Good performance
- Good off-the-shelf model in general for several scenarios
- Can approximate complex non-linear functions

## Cons 

- Long training time required
- Tuning required to determine optimal kernel for non-linear SVMs

## Requirements

- Scaled features
- Null values filled

## Resources

- [An Idiot's Guide to Support Vector Machines (SVMs) from MIT](https://web.mit.edu/6.034/wwwbob/svm.pdf)
- [Machine Learning Mastery's Post on Support Vector Machines for Machine Learning](https://machinelearningmastery.com/support-vector-machines-for-machine-learning/)